In [67]:
import numpy as np
from typing import Callable, TypeVar
from collections import Counter, deque, defaultdict
import itertools
from functools import cmp_to_key, cache
import regex as re
from intervaltree import Interval, IntervalTree
from concurrent.futures import ThreadPoolExecutor
from sortedcontainers import SortedList, SortedDict


np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

processors = {
  'int_list': lambda x: [int(y) for y in x.split()]
}

def search(start, get_neighbors, end_condition=lambda _, __: False, dfs=True):
    q, visited = deque([(start, 0)]), {}
    while q:
        current, distance = q.popleft() if dfs else q.pop()
        if end_condition(current, distance):
            return visited, current
        if current in visited:
            continue
        for node in get_neighbors(current, distance):
            q.append((node, distance+1))
        visited[current] = distance
    return visited, None

def debug_array(arr, coords):
    arr = arr.copy()
    for i in coords:
        arr[*i] = 'X'
    print(arr)

In [74]:

def day1():
    loc1, loc2 = zip(*data(1, processors['int_list']))
    part1 = sum(abs(x[0]-x[1]) for x in zip(sorted(loc1), sorted(loc2)))
    counts = Counter(loc2)
    part2 = sum(x*counts[x] for x in loc1)
    return part1, part2

day1()

(1941353, 22539317)

In [75]:
def day2():
    def check_safe(report):
        ascending = sorted(report)
        diffs = np.diff(ascending)
        return max(diffs) <= 3 and min(diffs) >= 1 and (
            report == ascending or
            report == list(reversed(ascending))
        )

    def check_safe_damp(report):
        if check_safe(report):
            return 1, 1
        for damped in itertools.combinations(report, len(report)-1):
            if check_safe(list(damped)):
                return 0, 1
        return 0, 0

    reports = data(2, processors['int_list'])
    safe = np.array((0,0))
    for report in reports:
        safe += check_safe_damp(report)
    return safe

day2()

array([356, 413])

In [76]:
def day3():
    def mul_strings(s):
        x, y = s.split(',')
        return int(x)*int(y)

    instructions = ''.join(data(3))
    matches = list(re.finditer(r'mul\((\d+,\d+)\)', instructions))
    conds = list(re.finditer(r"don't\(\).+?do\(\)", instructions))
    donts = IntervalTree([Interval(*cond.span()) for cond in conds])
    result = sum([mul_strings(mul[1]) * (1 if not donts[mul.span()[0]] else 1j) for mul in matches])
    return int(result.real+result.imag), int(result.real)

day3()

(182780583, 90772405)

In [75]:
def day4():
    grid = np.array(data(4, lambda x: np.array(list(x))))
    ymax, xmax = grid.shape

    def find_target_occurences(target):
        occurences = set()

        def get_neighbors(current, distance):
            col, row = int(current.real), int(current.imag)
            target_letter = target[distance]
            if grid[row, col] != target_letter:
                return
            if distance == len(target)-1:
                total.add(current)
                return
            for v in [1, -1, 1j, -1j, 1+1j, 1-1j, -1+1j, -1-1j]:
                new = current + v
                x, y = int(new.real), int(new.imag)
                if not (y >= 0 and x >= 0 and y < ymax and x < xmax):
                    continue
                yield new
        
        for j in range(ymax):
            for i in range(xmax):
                total = set()
                coordinate = i+1j*j
                search(coordinate, get_neighbors)
                for end in total:
                    occurences.add((coordinate, end))
        return occurences

    def find_diags(hits, l):
        centers = Counter()
        for start, end in hits:
            distance = end-start
            if abs(distance.real) == l and abs(distance.imag) == l:
                center = start + distance/2
                centers[center] += 1
        return centers
    
    def find_straights(hits, target):
        rev = target[::-1]
        l = len(target)
        td = l-1
        for start, end in hits:
            i, j = int(start.real), int(start.imag)
            d = end-start
            if (
                (d.real == td and not d.imag and ''.join(grid[j, i:i+l]) == target)
                or (d.real == -td and not d.imag and ''.join(grid[j, i-td:i+1]) == rev)
                or (not d.real and d.imag == td and ''.join(grid[j:j+l, i]) == target)
                or (not d.real and d.imag == -td and ''.join(grid[j-td:j+1, i]) == rev)
            ):
                yield start

    target = 'XMAS'
    matches = find_target_occurences(target)
    part1 = sum(find_diags(matches, len(target)-1).values()) + len(list(find_straights(matches, target)))

    centers = find_diags(find_target_occurences(target[1:]), len(target)-2)
    part2 = sum([1 if centers[x] == 2 else 0 for x in centers])

    return (part1, part2)

day4()

(2599, 1948)

In [78]:
def day5():
    text = data(5)
    split = text.index('')
    lists = [tuple(map(int, x.split(','))) for x in text[split+1:]]

    parents = defaultdict(lambda: set())
    for x in text[:split]:
        parent, child = tuple(map(int, x.split('|')))
        parents[child].add(parent)

    def check_illegal(nums):
        illegal = set()
        for num in nums:
            if num in illegal:
                return True
            illegal.update(parents[num])

    def compare(a, b):
        if a in parents[b]:
            return 1
        elif b in parents[a]:
            return -1
        return -1 if a < b else 1

    part1, part2 = 0, 0
    for nums in lists:
        if not check_illegal(nums):
            part1 += nums[len(nums)//2]
        else:
            part2 += sorted(nums, key=cmp_to_key(compare))[len(nums)//2]
            
    return part1, part2

day5()

(6041, 4884)

In [73]:
def day6():
    grid = np.array(data(0, list))
    ymax, xmax = grid.shape
    start = np.argwhere(grid == '^')[0]
    grid[*start] = '.'
    turns = [(1, 0), (0, 1), (-1, 0), (0, -1)]
    
    def run_guard(obstacle=(-1, -1)):
        directions, v = itertools.cycle(turns), turns[-1]
        y, x = int(start[0])-v[1], int(start[1])-v[0]
        visited, states = set(), set()
        while True:
            ny, nx = y+v[1], x + v[0]
            if (ny, nx, v) in states:
                return True, states
            elif (nx < 0 or ny < 0 or nx >= xmax or ny >= ymax):
                return False, visited
            elif grid[ny, nx] != '.' or (ny, nx) == obstacle:
                v = next(directions)
                continue
            y, x = ny, nx
            visited.add((y, x))
            states.add((y, x, v))

    _, original = run_guard()
    part1 = len(original)
    with ThreadPoolExecutor() as tpe:
        part2 = sum([r[0] for r in tpe.map(lambda x: run_guard(x), original)])
    return part1, part2

day6()

(41, 6)

In [11]:
def day7():
    equations = data(0, lambda x: [int(n) for n in re.split(r' |: ', x)])
    
    def check(n, acc, arr):
        if not arr:
            return n == acc
        elif acc > n:
            return False
        x, tail = arr[0], arr[1:]
        return (check(n, int(f'{acc}{x}'), tail) or
                check(n, acc*x, tail) or
                check(n, acc+x, tail))

    result = 0
    for eq in equations:
        if check(eq[0], eq[1], tuple(eq[2:])):
            result += eq[0]
    print(result)


day7()

11387


In [50]:
def day7():
    equations, flag = data(7, lambda x: [int(n) for n in re.split(r' |: ', x)]), False
    def check(n, acc, arr):
        if not arr:
            return n if n == acc else 0
        elif acc > n:
            return
        x, tail = arr[0], arr[1:]
        return ((flag and check(n, int(f'{acc}{x}'), tail)) or
                check(n, acc*x, tail) or
                check(n, acc+x, tail))
    part1, flag = sum([check(eq[0], eq[1], tuple(eq[2:])) for eq in equations]), True
    with ThreadPoolExecutor() as tpe:
        part2 = sum(tpe.map(lambda eq: check(eq[0], eq[1], tuple(eq[2:])), equations))
    return part1, part2

day7()

(7710205485870, 20928985450275)

In [72]:
def day8():
    grid = np.array(data(8, list))
    ymax, xmax = grid.shape
    def is_inside(y, x):
        return y >= 0 and y < ymax and x >= 0 and x < xmax
    def get_nodes(start, diff, sign=1):
        node = start.copy()
        while is_inside(*node):
            yield tuple(node)
            node -= sign*diff

    part1, part2 = set(), set()
    points = {i:np.argwhere(grid==i) for i in np.unique(grid) if i != '.'}
    for antennae in points:
        combos = itertools.combinations(points[antennae], 2)
        for combo in combos:
            diff = combo[1]-combo[0]
            for y, x in (combo[0]-diff, combo[1]+diff):
                if is_inside(y, x):
                    part1.add((y, x))
            for (y, x) in [*get_nodes(combo[0], diff), *get_nodes(combo[1], diff, -1)]:
                part2.add((y, x))

    return len(part1), len(part2)

day8()

(261, 898)

In [7]:
def sum_series(start, stop):
    n = (stop - start)
    sum = start + stop
    return n * sum // 2

In [39]:
def day9():
    disk_map = data(9)[0]
    print(disk_map)

    files = [int(x) for x in disk_map[0::2]]
    buffers = (int(x) for x in disk_map[1::2])
    print(len(files), files)
    
    p = 0 # Current pointer location
    buffer = 0 # Current buffer space available
    fid = 0 # Current file number from left to right
    checksum = 0 # Result

    for n in range(len(files)-1, -1, -1):
        req = files[n]
        while req > buffer:
            # fill buffer with rightmost file
            checksum += sum_series(p, p+buffer)*n
            p += buffer
            req -= buffer
            # add current file to checksum
            fsize = files[fid]
            checksum += sum_series(p, p+fsize)*fid
            p += fsize
            fid += 1
            if fid >= n:
                print('finish', n, 'p', p, 'req', req, 'fid', fid, 'checksum', checksum)
                checksum += sum_series(p, p+req)*fid
                return checksum
            # get next buffer
            buffer = next(buffers)
        checksum += sum_series(p, p+req)*n
        p += req
        buffer -= req

day9()

3862661670678085717870987066944388237122304387269526457922502763481789248860331275352261208057669164151786821267912912843756969762848733948147993167117987933260444111774981131046423127175391198132981556528886466841904153649781277634134462907446885521455261816827393693365787514157407057815067531283609939702491483692118848754199274930671713112463785536324065963870335246678963763726482884525850924546326162806111582168467928714746722135549182813311357913767784772057532215509849329438692631404536853075567677649217971898253752889665535729441681394169328065824974875236246829434715368379917054666169484640328312687686856934686335482475418948464312408116998373542665883885409969358775724793145487879242845351955713129714201538831838815217875978893510632238164136339832353066972996532394316253993598292538992698265645609135462016793077823110238350549616181835454778658492274752133885133617848131772264841284426255741812575847706824989369827790814863999436844435832394788125212563539120887592164845927156

6349606724455

In [86]:
def day9():
    disk_map = data(9)[0]

    files = [int(x) for x in disk_map[0::2]]
    buffers = (int(x) for x in disk_map[1::2])

    p = 0
    starts = {}
    slots = SortedDict()
    for i, v in enumerate(files):
        starts[i] = p
        p += v
        try:
            buffer = next(buffers)
            if buffer:
                slots[p] = buffer
                p += buffer
        except StopIteration:
            continue

    def find_slot(size):
        for index, value in slots.items():
            if value >= size:
                return index
        return -1

    checksum = 0
    for n in range(len(files)-1, -1, -1):
        req = files[n]
        p = find_slot(req)
        if p != -1 and p < starts[n]:
            buffer = slots[p]
            del slots[p]
            if (rem := buffer-req):
                slots[p+req] = rem
        else:
            p = starts[n]
        checksum += sum_series(p, p+req)*n
    return checksum

day9()

6376648986651

In [66]:
from sortedcontainers import SortedDict

a = SortedDict({'1':2, '3':0, '4':2})
a.popitem(), a.popitem(), a.popitem()

(('4', 2), ('3', 0), ('1', 2))